In [28]:
from drugsCom_raw.Utils import Utils
import  pandas as pd

all_conditions= pd.read_csv("outputs/Old/drugsComAll_final.csv")

utils = Utils()

In [29]:
conditions = utils.conditions_controlled_by_psychiatrists

In [30]:
selected_conditions = all_conditions[all_conditions.condition.isin(conditions)]

In [31]:
print(all_conditions.shape)
print(selected_conditions.shape)
print(selected_conditions.condition.unique().size)
print(all_conditions.condition.unique())

(212698, 10)
(50709, 10)
35
['Abnormal Uterine Bleeding' 'Amenorrhea' 'Birth Control' 'Endometriosis'
 'Endometrial Hyperplasia, Prophylaxis' 'Renal Cell Carcinoma'
 'Not Listed / Othe' 'Eye Redness' 'Benign Prostatic Hyperplasia' 'Acne'
 'ADHD' 'Depression' 'Narcolepsy' 'Allergic Rhinitis'
 'Interstitial Cystitis' 'Bladder Infection' 'Heart Attack'
 'High Blood Pressure' 'Angina' 'Rheumatoid Arthritis'
 'Chronic Lymphocytic Leukemia' "Non-Hodgkin's Lymphoma"
 'cal Segmental Glomerulosclerosis' 'Pemphigus' 'Edema' 'Pruritus' 'Pain'
 'Brain Tum' 'Breast Cance' 'Pancreatic Cance' 'Actinic Keratosis'
 'Gout, Acute' 'Bacterial Vaginitis' 'Rosacea' 'Perioral Dermatitis'
 'Migraine' 'Cluster Headaches' 'Cyclic Vomiting Syndrome'
 'New Daily Persistent Headache' 'Bacterial Skin Infection' 'Chronic Pain'
 'Back Pain' 'COPD' 'COPD, Maintenance' 'Irritable Bowel Syndrome'
 'Dysuria' 'Insomnia' 'Pulmonary Hypertension' 'Muscle Pain'
 'Osteoarthritis' "Behcet's Disease" 'Gout, Prophylaxis'
 'amili

In [32]:
latest_dataset = selected_conditions[["drugName","condition","text","rating","rating_label"]]

In [35]:
latest_dataset.to_csv(f"condition_drugName_label_text_dataset.csv", index=False)

781    I've been diagnosed with ADD at the age of 42 ...
Name: text, dtype: object

In [24]:
import re
from nltk import PorterStemmer
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

stemmer = PorterStemmer()
words = stopwords.words("english")
selected_conditions['cleaned'] = selected_conditions['text'].apply(lambda x: " ".join([stemmer.stem(i) for i in re.sub("[^a-zA-Z]", " ", x).split() if i not in words]).lower())

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/aliakkaya/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
/var/folders/n1/z4n1dv014gq2l7525zb6vs440000gn/T/ipykernel_253/55925020.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_conditions['cleaned'] = selected_conditions['text'].apply(lambda x: " ".join([stemmer.stem(i) for i in re.sub("[^a-zA-Z]", " ", x).split() if i not in words]).lower())


In [25]:
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest, chi2
import pickle
import numpy as np

In [26]:
selected_conditions.keys()

Index(['Unnamed: 0', 'drugName', 'condition', 'review', 'rating', 'date',
       'usefulCount', 'rating_label', 'plain_text', 'text', 'cleaned'],
      dtype='object')

In [27]:
vectorizer = TfidfVectorizer(min_df= 3, stop_words="english", sublinear_tf=True, norm='l2', ngram_range=(1, 3))
final_features = vectorizer.fit_transform(selected_conditions['cleaned']).toarray()

print("Features are done")



from sklearn.linear_model import LogisticRegression
X = selected_conditions['cleaned']
Y = selected_conditions['rating_label']
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.25)

pipeline = Pipeline([('vect', vectorizer),
                     ('chi',  SelectKBest(chi2, k=1200)),
                     ('clf', LogisticRegression(random_state=0))])

model = pipeline.fit(X_train, y_train)
with open('LogisticRegression.pickle', 'wb') as f:
    pickle.dump(model, f)

ytest = np.array(y_test)

# confusion matrix and classification report(precision, recall, F1-score)
print(classification_report(ytest, model.predict(X_test)))
print(confusion_matrix(ytest, model.predict(X_test)))

Features are done


/Users/aliakkaya/PycharmProjects/drugRepositioning/venv/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


              precision    recall  f1-score   support

          -1       0.72      0.47      0.57      2552
           0       0.00      0.00      0.00       990
           1       0.81      0.97      0.88      9136

    accuracy                           0.79     12678
   macro avg       0.51      0.48      0.48     12678
weighted avg       0.72      0.79      0.75     12678

[[1191    3 1358]
 [ 205    0  785]
 [ 264    1 8871]]
